In [2]:
import tensorflow as tf
tf.__version__
tf.config.experimental.enable_op_determinism()

In [3]:
import numpy as np
import pandas as pd
from sklearn.metrics import mean_absolute_error
import random

In [4]:
tf.keras.backend.clear_session()

In [5]:
def dfToInputRNN(df_input):
    reshaped_df_input = df_input.to_numpy()
    reshaped_df_input = reshaped_df_input.reshape(reshaped_df_input.shape[0], 1, reshaped_df_input.shape[1])
    return reshaped_df_input

In [6]:
subject = 'Piauí - Consumo de Cimento (t)'
split_index = 12 #Referente aos 13 anos de input  

In [7]:
data = pd.read_csv('../../Inputs Atuais/2008_01_model_input_PI.csv')
data = data[[col for col in data.columns if col != subject] + [subject]]
data

,Ano,Piauí - IDH Renda,Piauí - Desemprego,Piauí - IDH Educacao,Piauí - IDH Longevidade,Piauí - PIB - Estadual,Piauí - PIB - Construção Civil,Piauí - PIB - Per Capita,Piauí - PIB - Preços de Mercado,Piauí - Produção de Cimento (t),...,Taxa Selic (%),Piauí - IDH,Precipitation (mm/day),Atmospheric Pressure(mB),Global Radiation - Daylight (Kj/m²),Air Temperature - Dry Bulb (°C),Dew Point Temperature (°C),Air Relative Humidity (%),Wind Gust (m/s),Piauí - Consumo de Cimento (t)
0,2008,0.636377,7.951466,0.633132,0.744861,2.296156e+07,1.400359e+06,6.253673,1.932330e+07,510.644488,...,11.760983,0.670241,24.296582,1002.253734,1279.489487,28.374914,21.343931,67.378713,1.314599,449.076000
1,2009,0.638637,7.981121,0.637582,0.746332,2.318896e+07,1.418401e+06,6.277339,1.939333e+07,564.654167,...,9.472585,0.672829,23.274048,1002.162518,1397.889138,27.971629,21.269184,69.377054,1.242652,536.766000
2,2010,0.640114,7.946167,0.641655,0.747336,2.385426e+07,1.441334e+06,6.336644,1.960635e+07,595.360558,...,9.332485,0.674904,19.182853,1002.283003,1437.681759,27.813041,21.237580,70.062576,1.227247,639.535000
3,2011,0.640823,7.934312,0.645347,0.747880,2.447423e+07,1.448568e+06,6.385667,1.978514e+07,612.063590,...,10.985689,0.676479,18.107389,1002.395371,1443.014477,27.920185,21.009865,69.079510,1.247622,625.022000
4,2012,0.640158,9.098665,0.648074,0.747956,2.502337e+07,1.450923e+06,6.419744,1.991803e+07,616.463496,...,8.159014,0.677105,18.252844,1002.464093,1406.026571,27.983708,20.651339,67.998886,1.231493,687.536000
5,2013,0.640588,9.874811,0.652080,0.747535,2.552759e+07,1.456057e+06,6.451661,2.003921e+07,585.346961,...,7.913561,0.678531,18.670752,1002.768122,1348.345803,27.951493,20.492168,67.705538,1.231771,778.779000
6,2014,0.638470,10.508945,0.654503,0.746830,2.597493e+07,1.458827e+06,6.481158,2.014649e+07,534.569599,...,10.350442,0.678492,18.366943,1002.930152,1312.189559,28.101214,20.242493,66.500996,1.263497,755.713857
7,2015,0.636259,10.965401,0.655795,0.745732,2.623707e+07,1.443457e+06,6.481716,2.015777e+07,485.544522,...,12.454398,0.677866,19.175926,1002.890504,1261.987570,28.208348,20.043769,65.528148,1.276249,794.945486
8,2016,0.648680,11.203809,0.670587,0.753641,2.645308e+07,1.433449e+06,6.489827,2.018916e+07,383.195013,...,13.111908,0.689454,22.137114,1002.832338,1235.205362,28.178052,20.095966,65.401854,1.267215,842.687657
9,2017,0.648105,11.173036,0.673185,0.752518,2.658816e+07,1.433526e+06,6.502281,2.023043e+07,274.263754,...,9.479812,0.689859,24.264986,1002.920008,1214.156503,28.068432,20.444122,66.544636,1.228392,831.356000


In [8]:
input_data = data.iloc[:-2, 1:-1]
input_data

,Piauí - IDH Renda,Piauí - Desemprego,Piauí - IDH Educacao,Piauí - IDH Longevidade,Piauí - PIB - Estadual,Piauí - PIB - Construção Civil,Piauí - PIB - Per Capita,Piauí - PIB - Preços de Mercado,Piauí - Produção de Cimento (t),Contratação Comercial (Bi R$),...,NFSP - Porcentagem do PIB (%),Taxa Selic (%),Piauí - IDH,Precipitation (mm/day),Atmospheric Pressure(mB),Global Radiation - Daylight (Kj/m²),Air Temperature - Dry Bulb (°C),Dew Point Temperature (°C),Air Relative Humidity (%),Wind Gust (m/s)
0,0.636377,7.951466,0.633132,0.744861,2.296156e+07,1.400359e+06,6.253673,1.932330e+07,510.644488,5.483335e+08,...,2.868108,11.760983,0.670241,24.296582,1002.253734,1279.489487,28.374914,21.343931,67.378713,1.314599
1,0.638637,7.981121,0.637582,0.746332,2.318896e+07,1.418401e+06,6.277339,1.939333e+07,564.654167,5.302764e+08,...,2.305885,9.472585,0.672829,23.274048,1002.162518,1397.889138,27.971629,21.269184,69.377054,1.242652
2,0.640114,7.946167,0.641655,0.747336,2.385426e+07,1.441334e+06,6.336644,1.960635e+07,595.360558,5.046425e+08,...,1.958246,9.332485,0.674904,19.182853,1002.283003,1437.681759,27.813041,21.237580,70.062576,1.227247
3,0.640823,7.934312,0.645347,0.747880,2.447423e+07,1.448568e+06,6.385667,1.978514e+07,612.063590,4.854622e+08,...,2.079088,10.985689,0.676479,18.107389,1002.395371,1443.014477,27.920185,21.009865,69.079510,1.247622
4,0.640158,9.098665,0.648074,0.747956,2.502337e+07,1.450923e+06,6.419744,1.991803e+07,616.463496,4.867661e+08,...,2.183282,8.159014,0.677105,18.252844,1002.464093,1406.026571,27.983708,20.651339,67.998886,1.231493
5,0.640588,9.874811,0.652080,0.747535,2.552759e+07,1.456057e+06,6.451661,2.003921e+07,585.346961,5.225848e+08,...,2.403445,7.913561,0.678531,18.670752,1002.768122,1348.345803,27.951493,20.492168,67.705538,1.231771
6,0.638470,10.508945,0.654503,0.746830,2.597493e+07,1.458827e+06,6.481158,2.014649e+07,534.569599,5.676413e+08,...,2.748397,10.350442,0.678492,18.366943,1002.930152,1312.189559,28.101214,20.242493,66.500996,1.263497
7,0.636259,10.965401,0.655795,0.745732,2.623707e+07,1.443457e+06,6.481716,2.015777e+07,485.544522,5.179392e+08,...,2.501529,12.454398,0.677866,19.175926,1002.890504,1261.987570,28.208348,20.043769,65.528148,1.276249
8,0.648680,11.203809,0.670587,0.753641,2.645308e+07,1.433449e+06,6.489827,2.018916e+07,383.195013,4.524144e+08,...,2.073464,13.111908,0.689454,22.137114,1002.832338,1235.205362,28.178052,20.095966,65.401854,1.267215
9,0.648105,11.173036,0.673185,0.752518,2.658816e+07,1.433526e+06,6.502281,2.023043e+07,274.263754,4.213689e+08,...,2.177386,9.479812,0.689859,24.264986,1002.920008,1214.156503,28.068432,20.444122,66.544636,1.228392


In [9]:
# Shift para prever futuro e não presente
target_data = data[subject].shift(-1)
target_data

0     536.766000
1     639.535000
2     625.022000
3     687.536000
4     778.779000
5     755.713857
6     794.945486
7     842.687657
8     831.356000
9     626.223000
10    702.437000
11    788.164000
12    826.368000
13    804.417000
14           NaN
Name: Piauí - Consumo de Cimento (t), dtype: float64

In [10]:
# input para treinamento
train_input = (input_data.iloc[:-1] - np.mean(input_data.iloc[:-1], axis=0)) / np.std(input_data.iloc[:-1], axis=0)
train_input

,Piauí - IDH Renda,Piauí - Desemprego,Piauí - IDH Educacao,Piauí - IDH Longevidade,Piauí - PIB - Estadual,Piauí - PIB - Construção Civil,Piauí - PIB - Per Capita,Piauí - PIB - Preços de Mercado,Piauí - Produção de Cimento (t),Contratação Comercial (Bi R$),...,NFSP - Porcentagem do PIB (%),Taxa Selic (%),Piauí - IDH,Precipitation (mm/day),Atmospheric Pressure(mB),Global Radiation - Daylight (Kj/m²),Air Temperature - Dry Bulb (°C),Dew Point Temperature (°C),Air Relative Humidity (%),Wind Gust (m/s)
0,-1.244793,-1.286179,-1.543242,-1.364689,-1.791927,-2.430395,-1.983161,-1.934577,0.310888,1.204752,...,2.026352,0.984445,-1.486495,1.532542,-1.329511,-0.499138,1.719528,1.574885,0.008389,2.526713
1,-0.733935,-1.264664,-1.229292,-0.796636,-1.615264,-1.276024,-1.702934,-1.710653,0.665029,0.820657,...,0.085558,-0.051710,-1.104860,1.094793,-1.637011,0.971937,-0.703092,1.403767,1.371924,-0.247365
2,-0.399968,-1.290024,-0.941897,-0.409264,-1.098406,0.191239,-1.000708,-1.029511,0.866371,0.275392,...,-1.114490,-0.115145,-0.798779,-0.656658,-1.230840,1.466347,-1.655760,1.331414,1.839678,-0.841315
3,-0.239582,-1.298625,-0.681446,-0.199210,-0.616760,0.654083,-0.420225,-0.457837,0.975892,-0.132596,...,-0.697344,0.633403,-0.566581,-1.117067,-0.852032,1.532604,-1.012123,0.810102,1.168900,-0.055737
4,-0.389944,-0.453869,-0.489065,-0.170018,-0.190143,0.804775,-0.016723,-0.032922,1.004742,-0.104860,...,-0.337667,-0.646476,-0.474249,-1.054798,-0.620359,1.073042,-0.630530,-0.010676,0.431554,-0.677618
5,-0.292755,0.109237,-0.206415,-0.332297,0.201584,1.133256,0.361197,0.354549,0.800712,0.657046,...,0.422337,-0.757614,-0.264062,-0.875890,0.404563,0.356378,-0.824051,-0.375069,0.231393,-0.666887
6,-0.771591,0.569311,-0.035440,-0.604609,0.549114,1.310516,0.710469,0.697572,0.467765,1.615453,...,1.613108,0.345772,-0.269828,-1.005951,0.950789,-0.092851,0.075353,-0.946654,-0.590506,0.556362
7,-1.271487,0.900476,0.055711,-1.028293,0.752766,0.327112,0.717082,0.733641,0.146308,0.558230,...,0.760923,1.298413,-0.362089,-0.659624,0.817133,-0.716594,0.718932,-1.401597,-1.254312,1.048065
8,1.536902,1.073445,1.099309,2.024843,0.920581,-0.313247,0.813122,0.834017,-0.524796,-0.835563,...,-0.716758,1.596124,1.346608,0.608069,0.621045,-1.049353,0.536936,-1.282101,-1.340487,0.699728
9,1.406981,1.051119,1.282663,1.591118,1.025520,-0.308314,0.960590,0.965973,-1.239057,-1.495939,...,-0.358019,-0.048438,1.406313,1.519016,0.916592,-1.310878,-0.121575,-0.485063,-0.560729,-0.797166


In [11]:
reshaped_train_input = dfToInputRNN(train_input)

In [12]:
validation_input = ((input_data - np.mean(input_data, axis=0)) / np.std(input_data, axis=0)).iloc[split_index - 1:-1]
validation_input = dfToInputRNN(validation_input)
validation_input

array([[[ 1.03720327,  0.88451686,  1.23879267,  0.33319037,
          0.82904731,  0.07834927,  0.64873804,  0.6622595 ,
         -1.54905892, -1.0453709 ,  1.21859064, -0.99976018,
          0.79428371,  1.15575749, -0.52559858,  1.30260559,
         -0.55253046, -0.25130359,  1.38787176,  1.47154451,
         -1.80649616,  1.09296291,  0.94281138,  0.52836084,
          0.61258103,  0.87656257,  1.47801536,  1.71982086,
          1.28221181,  1.39086472,  1.58209844,  1.08872091,
          0.55212962, -1.81398118,  1.44113674,  1.11191094,
          1.30470835,  0.86390516, -1.67365968, -1.70738939,
         -1.55813196, -0.48498301, -0.75714639,  0.40770893,
         -0.27394734,  2.02323219,  1.2908951 ,  1.70465513,
          1.81148084,  1.39968653, -1.36191442,  1.28024593,
          1.11054795,  0.12662799,  1.40414112,  1.40468999,
          1.45643959,  1.58509336,  1.32283157,  1.30222047,
          1.55138602,  1.91801602, -0.77489784,  2.32098884,
         -1.1942541 , -1

In [13]:
# Alvo para treinamento
train_target = target_data.iloc[:-3]
train_target

0     536.766000
1     639.535000
2     625.022000
3     687.536000
4     778.779000
5     755.713857
6     794.945486
7     842.687657
8     831.356000
9     626.223000
10    702.437000
11    788.164000
Name: Piauí - Consumo de Cimento (t), dtype: float64

In [14]:
# Alvo para validação do treinamento
validation_target = target_data.iloc[split_index - 1:-3]
validation_target

11    788.164
Name: Piauí - Consumo de Cimento (t), dtype: float64

In [15]:
#input de test (Ano 2021)
test_input = ((input_data - np.mean(input_data, axis=0)) / np.std(input_data, axis=0)).iloc[split_index:]
test_input

,Piauí - IDH Renda,Piauí - Desemprego,Piauí - IDH Educacao,Piauí - IDH Longevidade,Piauí - PIB - Estadual,Piauí - PIB - Construção Civil,Piauí - PIB - Per Capita,Piauí - PIB - Preços de Mercado,Piauí - Produção de Cimento (t),Contratação Comercial (Bi R$),...,NFSP - Porcentagem do PIB (%),Taxa Selic (%),Piauí - IDH,Precipitation (mm/day),Atmospheric Pressure(mB),Global Radiation - Daylight (Kj/m²),Air Temperature - Dry Bulb (°C),Dew Point Temperature (°C),Air Relative Humidity (%),Wind Gust (m/s)
12,0.823567,0.785725,1.126392,-0.598225,0.529639,0.378729,0.253155,0.276615,-1.651728,1.108405,...,-1.482779,-2.260447,0.91739,1.011919,0.418013,-0.843464,1.091606,-0.183281,-0.442768,-1.555022


In [16]:
test_input = test_input.to_numpy()
reshaped_test_input = test_input.reshape(test_input.shape[0], 1, test_input.shape[1])
reshaped_test_input

array([[[ 0.82356725,  0.78572538,  1.12639184, -0.5982253 ,
          0.52963862,  0.37872907,  0.25315465,  0.27661526,
         -1.65172808,  1.10840491,  2.63582309, -0.18995911,
          0.72640489,  2.66833308, -0.032632  ,  0.98802435,
          0.24980494, -0.08955808,  1.85434394,  2.02706568,
         -2.21150703,  1.42938591,  0.85132668,  0.14092386,
          0.30902324,  1.00267904,  2.27970137,  1.11272593,
          0.61007902,  0.89064538,  1.42231438,  1.02650562,
          1.47127593, -1.53201473,  1.73503858,  1.66610613,
          1.0456894 ,  0.55289259, -1.81216593, -2.10342296,
         -2.55377478,  0.41940628, -1.75756445,  0.241023  ,
         -0.98659345,  1.52184916,  1.01398607,  1.49290967,
          1.40764092,  1.87930411, -2.51892072,  2.83356214,
          1.58620924, -0.02524128,  1.54231542,  1.5176119 ,
          1.7355269 ,  1.87941982,  1.54198129,  1.51276712,
          1.83795594,  2.62210693,  1.04993174,  0.96476493,
         -1.48277861, -2

In [17]:
# Alvo de test (Ano 2021)
test_target = target_data.iloc[split_index:-2]
test_target

12    826.368
Name: Piauí - Consumo de Cimento (t), dtype: float64

In [18]:
# Rede Neural Recorrente com optmizador Estocástico
def bidirectional_lstm_model(train_input, train_target, want_verbose=1, seed=0):
    if seed != 0:
        random.seed(seed)
        np.random.seed(seed)
        tf.random.set_seed(seed)
        tf.keras.utils.set_random_seed(seed)

    # Aṕos 500 epochs sem grandes melhoras no val_loss, interrompe.
    early_stopping = tf.keras.callbacks.EarlyStopping(monitor='loss', 
                                                      patience=500, 
                                                      verbose=want_verbose, 
                                                      restore_best_weights=True,
                                                      start_from_epoch=500)
    # Método estocástico e learning rate=0.01
    optimizer = tf.keras.optimizers.SGD(learning_rate=0.01)
    model = tf.keras.Sequential([
        tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(296, activation='tanh', return_sequences=True), 
                                      input_shape=(train_input.shape[1], train_input.shape[2])),
        tf.keras.layers.Dropout(0.5),
        tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(148, activation='tanh')),
        tf.keras.layers.Dropout(0.2),
        tf.keras.layers.Dense(1)
    ])
    model.compile(optimizer=optimizer, loss='mean_squared_error')   
    history = model.fit(train_input, 
                        train_target, 
                        epochs=10000,
                        batch_size=train_input.shape[0],
                        callbacks=[early_stopping], 
                        verbose=want_verbose)
    return model, history

In [19]:
def get_a_good_seed(train_input, train_target):

    random_seeds = [random.randint(0, 2**32 - 1) for _ in range(100)]
    print(random_seeds)

    best_loss = float('inf')
    winner_seed = None
    i = 0
    for seed in random_seeds:
        print(f"\n\nStep: {i} ___________________________________________")
        i += 1

        model, history = bidirectional_lstm_model(train_input, train_target, want_verbose=0, seed=seed)
        current_loss = min(history.history['loss'][500:])
        print(f"loss: {current_loss}")

        if current_loss < best_loss:
            best_loss = current_loss
            winner_seed = seed
            print(f"winner_seed: {winner_seed}")

    return winner_seed

In [21]:
winner_seed = get_a_good_seed(reshaped_train_input, train_target)
print(f"\n\nfinal_seed: {winner_seed}")

[3377070277, 4148746486, 1444802088, 20361787, 1091267669, 3101682604, 2762453560, 2203652460, 627709391, 820141846, 849055210, 2737495382, 1497487385, 1645722673, 424457709, 195012299, 3008425469, 1932411114, 1664057643, 3036857502, 81548222, 2059248458, 1334290571, 253403516, 3321050136, 134966432, 2270572290, 546163170, 1979410272, 2418416451, 3951735825, 153496935, 309191791, 271841609, 2678690614, 829571372, 4179723712, 4126910646, 116722105, 4099230642, 1212650391, 652958253, 1115791358, 4064321447, 4077925683, 3429341675, 3887168661, 3727543773, 3954636909, 2277186523, 522623884, 497794688, 438827650, 37493368, 3727187320, 1360216357, 428320381, 133015262, 1549783853, 2875740100, 682551499, 1979741914, 4000405845, 515677777, 3189089567, 207685054, 1503150732, 4083762319, 1657756373, 1407320159, 3162731791, 554854212, 2288425575, 564442677, 3041932264, 239771691, 4161137285, 2869222499, 1442259142, 1465287971, 1384395524, 2086328907, 3753418913, 3448473426, 768610629, 3403520348,

2023-09-09 15:25:23.939154: I tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:432] Loaded cuDNN version 8600
2023-09-09 15:25:24.051196: I tensorflow/compiler/xla/stream_executor/cuda/cuda_blas.cc:606] TensorFloat-32 will be used for the matrix multiplication. This will only be logged once.
2023-09-09 15:25:24.062438: I tensorflow/compiler/xla/service/service.cc:168] XLA service 0x7f6aa8458fa0 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
2023-09-09 15:25:24.062450: I tensorflow/compiler/xla/service/service.cc:176]   StreamExecutor device (0): NVIDIA GeForce RTX 3060, Compute Capability 8.6
2023-09-09 15:25:24.311172: I ./tensorflow/compiler/jit/device_compiler.h:186] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


loss: 302.1582946777344
winner_seed: 3377070277


Step: 1 ___________________________________________
loss: 399.0806579589844


Step: 2 ___________________________________________
loss: 477.0258483886719


Step: 3 ___________________________________________
loss: 353.6879577636719


Step: 4 ___________________________________________
loss: 444.62451171875


Step: 5 ___________________________________________
loss: 457.0702819824219


Step: 6 ___________________________________________
loss: 171.8524932861328
winner_seed: 2762453560


Step: 7 ___________________________________________
loss: 270.0617980957031


Step: 8 ___________________________________________
loss: 319.6564025878906


Step: 9 ___________________________________________
loss: 566.8726196289062


Step: 10 ___________________________________________
loss: 482.3709716796875


Step: 11 ___________________________________________
loss: 223.64239501953125


Step: 12 ___________________________________________
loss: 532.8136

In [22]:
trained_model, history = bidirectional_lstm_model(reshaped_train_input, 
                                                  train_target, want_verbose=1, seed=winner_seed)

Epoch 1/10000
1/1 [==============================] - 2s 2s/step - loss: 523171.0938
Epoch 2/10000
1/1 [==============================] - 0s 4ms/step - loss: 499237.3438
Epoch 3/10000
1/1 [==============================] - 0s 4ms/step - loss: 354655.5312
Epoch 4/10000
1/1 [==============================] - 0s 3ms/step - loss: 31633.3594
Epoch 5/10000
1/1 [==============================] - 0s 4ms/step - loss: 114124.2109
Epoch 6/10000
1/1 [==============================] - 0s 4ms/step - loss: 77051.2734
Epoch 7/10000
1/1 [==============================] - 0s 4ms/step - loss: 85849.4453
Epoch 8/10000
1/1 [==============================] - 0s 4ms/step - loss: 50972.3008
Epoch 9/10000
1/1 [==============================] - 0s 4ms/step - loss: 32228.4316
Epoch 10/10000
1/1 [==============================] - 0s 4ms/step - loss: 20002.5918
Epoch 11/10000
1/1 [==============================] - 0s 4ms/step - loss: 15264.3311
Epoch 12/10000
1/1 [==============================] - 0s 3ms/step - los

1/1 [==============================] - 0s 4ms/step - loss: 12229.2627
Epoch 99/10000
1/1 [==============================] - 0s 4ms/step - loss: 6334.9224
Epoch 100/10000
1/1 [==============================] - 0s 4ms/step - loss: 5374.8789
Epoch 101/10000
1/1 [==============================] - 0s 4ms/step - loss: 7059.3540
Epoch 102/10000
1/1 [==============================] - 0s 4ms/step - loss: 4203.4312
Epoch 103/10000
1/1 [==============================] - 0s 4ms/step - loss: 3359.1575
Epoch 104/10000
1/1 [==============================] - 0s 4ms/step - loss: 2342.0623
Epoch 105/10000
1/1 [==============================] - 0s 4ms/step - loss: 3334.9451
Epoch 106/10000
1/1 [==============================] - 0s 4ms/step - loss: 3455.8210
Epoch 107/10000
1/1 [==============================] - 0s 4ms/step - loss: 2579.1658
Epoch 108/10000
1/1 [==============================] - 0s 4ms/step - loss: 4741.5259
Epoch 109/10000
1/1 [==============================] - 0s 3ms/step - loss: 5208.9

1/1 [==============================] - 0s 4ms/step - loss: 1831.7168
Epoch 195/10000
1/1 [==============================] - 0s 4ms/step - loss: 1966.2520
Epoch 196/10000
1/1 [==============================] - 0s 3ms/step - loss: 2370.1284
Epoch 197/10000
1/1 [==============================] - 0s 4ms/step - loss: 4354.3472
Epoch 198/10000
1/1 [==============================] - 0s 4ms/step - loss: 4307.6265
Epoch 199/10000
1/1 [==============================] - 0s 4ms/step - loss: 4870.4146
Epoch 200/10000
1/1 [==============================] - 0s 4ms/step - loss: 3998.6438
Epoch 201/10000
1/1 [==============================] - 0s 4ms/step - loss: 1453.7067
Epoch 202/10000
1/1 [==============================] - 0s 4ms/step - loss: 2854.3164
Epoch 203/10000
1/1 [==============================] - 0s 4ms/step - loss: 3270.5576
Epoch 204/10000
1/1 [==============================] - 0s 4ms/step - loss: 3505.6414
Epoch 205/10000
1/1 [==============================] - 0s 3ms/step - loss: 1604.6

1/1 [==============================] - 0s 4ms/step - loss: 2742.4363
Epoch 291/10000
1/1 [==============================] - 0s 4ms/step - loss: 2508.0977
Epoch 292/10000
1/1 [==============================] - 0s 3ms/step - loss: 2866.4670
Epoch 293/10000
1/1 [==============================] - 0s 4ms/step - loss: 4471.7441
Epoch 294/10000
1/1 [==============================] - 0s 4ms/step - loss: 3390.5090
Epoch 295/10000
1/1 [==============================] - 0s 3ms/step - loss: 1886.8893
Epoch 296/10000
1/1 [==============================] - 0s 4ms/step - loss: 3449.1389
Epoch 297/10000
1/1 [==============================] - 0s 4ms/step - loss: 2679.8621
Epoch 298/10000
1/1 [==============================] - 0s 4ms/step - loss: 3259.1045
Epoch 299/10000
1/1 [==============================] - 0s 4ms/step - loss: 3527.9688
Epoch 300/10000
1/1 [==============================] - 0s 4ms/step - loss: 3643.9475
Epoch 301/10000
1/1 [==============================] - 0s 5ms/step - loss: 1686.0

1/1 [==============================] - 0s 4ms/step - loss: 1422.7467
Epoch 387/10000
1/1 [==============================] - 0s 4ms/step - loss: 2973.6416
Epoch 388/10000
1/1 [==============================] - 0s 4ms/step - loss: 3330.0039
Epoch 389/10000
1/1 [==============================] - 0s 4ms/step - loss: 3386.4324
Epoch 390/10000
1/1 [==============================] - 0s 4ms/step - loss: 3562.2551
Epoch 391/10000
1/1 [==============================] - 0s 4ms/step - loss: 2923.3679
Epoch 392/10000
1/1 [==============================] - 0s 4ms/step - loss: 1350.1906
Epoch 393/10000
1/1 [==============================] - 0s 4ms/step - loss: 3381.3914
Epoch 394/10000
1/1 [==============================] - 0s 4ms/step - loss: 1515.0204
Epoch 395/10000
1/1 [==============================] - 0s 4ms/step - loss: 2205.5989
Epoch 396/10000
1/1 [==============================] - 0s 4ms/step - loss: 3196.1162
Epoch 397/10000
1/1 [==============================] - 0s 4ms/step - loss: 3667.4

...

1/1 [==============================] - 0s 4ms/step - loss: 2034.8590
Epoch 1819/10000
1/1 [==============================] - 0s 4ms/step - loss: 1067.0282
Epoch 1820/10000
1/1 [==============================] - 0s 5ms/step - loss: 2099.4685
Epoch 1821/10000
1/1 [==============================] - 0s 4ms/step - loss: 1457.4082
Epoch 1822/10000
1/1 [==============================] - 0s 4ms/step - loss: 1085.5376
Epoch 1823/10000
1/1 [==============================] - 0s 4ms/step - loss: 3802.7129
Epoch 1824/10000
1/1 [==============================] - 0s 4ms/step - loss: 3198.9099
Epoch 1825/10000
1/1 [==============================] - 0s 4ms/step - loss: 1366.5646
Epoch 1826/10000
1/1 [==============================] - 0s 4ms/step - loss: 1556.3315
Epoch 1827/10000
1/1 [==============================] - 0s 4ms/step - loss: 1371.3939
Epoch 1828/10000
1/1 [==============================] - 0s 4ms/step - loss: 2453.5593
Epoch 1829/10000
1/1 [==============================] - 0s 4ms/step - l

1/1 [==============================] - 0s 4ms/step - loss: 855.7186
Epoch 1914/10000
1/1 [==============================] - 0s 4ms/step - loss: 1569.7452
Epoch 1915/10000
1/1 [==============================] - 0s 4ms/step - loss: 1770.4303
Epoch 1916/10000
1/1 [==============================] - 0s 4ms/step - loss: 2955.1687
Epoch 1917/10000
1/1 [==============================] - 0s 3ms/step - loss: 1928.2275
Epoch 1918/10000
1/1 [==============================] - 0s 4ms/step - loss: 1882.4789
Epoch 1919/10000
1/1 [==============================] - 0s 4ms/step - loss: 784.8176
Epoch 1920/10000
1/1 [==============================] - 0s 4ms/step - loss: 2507.7517
Epoch 1921/10000
1/1 [==============================] - 0s 4ms/step - loss: 3215.2507
Epoch 1922/10000
1/1 [==============================] - 0s 4ms/step - loss: 2415.8381
Epoch 1923/10000
1/1 [==============================] - 0s 4ms/step - loss: 1211.2311
Epoch 1924/10000
1/1 [==============================] - 0s 4ms/step - los

1/1 [==============================] - 0s 4ms/step - loss: 1052.7650
Epoch 2009/10000
1/1 [==============================] - 0s 4ms/step - loss: 1222.0074
Epoch 2010/10000
1/1 [==============================] - 0s 4ms/step - loss: 2096.3994
Epoch 2011/10000
1/1 [==============================] - 0s 4ms/step - loss: 2013.3180
Epoch 2012/10000
1/1 [==============================] - 0s 3ms/step - loss: 1743.9364
Epoch 2013/10000
1/1 [==============================] - 0s 4ms/step - loss: 2061.7415
Epoch 2014/10000
1/1 [==============================] - 0s 4ms/step - loss: 3181.3154
Epoch 2015/10000
1/1 [==============================] - 0s 4ms/step - loss: 2876.2209
Epoch 2016/10000
1/1 [==============================] - 0s 4ms/step - loss: 1349.7266
Epoch 2017/10000
1/1 [==============================] - 0s 4ms/step - loss: 2385.9216
Epoch 2018/10000
1/1 [==============================] - 0s 4ms/step - loss: 1793.0215
Epoch 2019/10000
1/1 [==============================] - 0s 4ms/step - l

1/1 [==============================] - 0s 4ms/step - loss: 3048.0920
Epoch 2104/10000
1/1 [==============================] - 0s 4ms/step - loss: 1670.0537
Epoch 2105/10000
1/1 [==============================] - 0s 4ms/step - loss: 2766.1914
Epoch 2106/10000
1/1 [==============================] - 0s 4ms/step - loss: 1324.9542
Epoch 2107/10000
1/1 [==============================] - 0s 4ms/step - loss: 1909.1530
Epoch 2108/10000
1/1 [==============================] - 0s 4ms/step - loss: 971.8787
Epoch 2109/10000
1/1 [==============================] - 0s 5ms/step - loss: 983.4488
Epoch 2110/10000
1/1 [==============================] - 0s 4ms/step - loss: 3371.8691
Epoch 2111/10000
1/1 [==============================] - 0s 4ms/step - loss: 984.3328
Epoch 2112/10000
1/1 [==============================] - 0s 4ms/step - loss: 2520.6292
Epoch 2113/10000
1/1 [==============================] - 0s 4ms/step - loss: 1380.6680
Epoch 2114/10000
1/1 [==============================] - 0s 4ms/step - loss

1/1 [==============================] - 0s 4ms/step - loss: 1351.7626
Epoch 2199/10000
1/1 [==============================] - 0s 4ms/step - loss: 1233.0469
Epoch 2200/10000
1/1 [==============================] - 0s 4ms/step - loss: 2747.4512
Epoch 2201/10000
1/1 [==============================] - 0s 4ms/step - loss: 2781.1858
Epoch 2202/10000
1/1 [==============================] - 0s 4ms/step - loss: 2303.0293
Epoch 2203/10000
1/1 [==============================] - 0s 3ms/step - loss: 2159.7366
Epoch 2204/10000
1/1 [==============================] - 0s 3ms/step - loss: 2562.2546
Epoch 2205/10000
1/1 [==============================] - 0s 3ms/step - loss: 2143.6628
Epoch 2206/10000
1/1 [==============================] - 0s 4ms/step - loss: 1534.7241
Epoch 2207/10000
1/1 [==============================] - 0s 3ms/step - loss: 1615.1835
Epoch 2208/10000
1/1 [==============================] - 0s 3ms/step - loss: 2463.6799
Epoch 2209/10000
1/1 [==============================] - 0s 3ms/step - l

In [23]:
prediction = trained_model.predict(reshaped_test_input)[0]
print(f"{test_target.values}(test_target) - {prediction}(prediction) = {mean_absolute_error(test_target, prediction)}")

1/1 [==============================] - 1s 599ms/step
[826.368](test_target) - [774.4936](prediction) = 51.8744086914063


In [24]:
porcentage = mean_absolute_error(test_target, prediction)/test_target.values
porcentage[0]

0.06277398046803155

In [25]:
def model_time_series_mae_tester(data, window, seed):
    if window > 2:

        # Copia e organiza data geral
        predictions = []
        targets = []
        data_copy = data.copy()
        data_copy[subject] = data[subject].shift(-1)
        data_copy = data_copy.iloc[:-1,1:]
        
        input_df = data_copy.iloc[:, :-1]
        target_df = data_copy.iloc[:, -1:]

        for i in range(window, len(data_copy) + 1):

            train_input = input_df.iloc[:i - 1]
            train_input = (train_input - np.mean(train_input, axis=0)) / np.std(train_input, axis=0)
            print("train_input:")
            display(train_input)

            train_target = target_df.iloc[:i - 1]
            print("train_target:")
            display(train_target)

            test_input = input_df.iloc[:i]
            test_input = ((test_input - np.mean(test_input, axis=0)) / np.std(test_input, axis=0)).iloc[-1:]
            print("test_input:")
            display(test_input)

            test_target = target_df.iloc[i - 1:i]
            print("test_target:")
            display(test_target)
            
            model, hitory = bidirectional_lstm_model(dfToInputRNN(train_input), 
                                                     train_target, want_verbose=0, seed=seed)

            prediction = model.predict(dfToInputRNN(test_input))[0]

            current_error = mean_absolute_error(test_target, prediction)
            print(f"Error: {current_error}\n\n")

            predictions.append(prediction[0].item())
            targets.append(test_target.iloc[0, 0])

        return predictions, targets

In [26]:
predictions, targets = model_time_series_mae_tester(data.iloc[:-1,:], 4, winner_seed)
predictions

train_input:


,Piauí - IDH Renda,Piauí - Desemprego,Piauí - IDH Educacao,Piauí - IDH Longevidade,Piauí - PIB - Estadual,Piauí - PIB - Construção Civil,Piauí - PIB - Per Capita,Piauí - PIB - Preços de Mercado,Piauí - Produção de Cimento (t),Contratação Comercial (Bi R$),...,NFSP - Porcentagem do PIB (%),Taxa Selic (%),Piauí - IDH,Precipitation (mm/day),Atmospheric Pressure(mB),Global Radiation - Daylight (Kj/m²),Air Temperature - Dry Bulb (°C),Dew Point Temperature (°C),Air Relative Humidity (%),Wind Gust (m/s)
0,-1.300755,-0.527817,-1.242369,-1.294258,-0.985718,-1.173238,-1.018535,-0.977767,-1.320495,1.148205,...,1.308949,1.412346,-1.267049,0.925774,0.402439,-1.372251,1.360201,1.353721,-1.370823,1.394785
1,0.169724,1.400155,0.036043,0.153497,-0.385361,-0.097218,-0.340403,-0.395977,0.221818,0.140893,...,-0.190804,-0.643250,0.089521,0.462967,-1.375329,0.389992,-0.344854,-0.322487,0.384358,-0.495076
2,1.131031,-0.872338,1.206325,1.140761,1.371079,1.270457,1.358938,1.373744,1.098677,-1.289098,...,-1.118144,-0.769096,1.177528,-1.388742,0.972890,0.982259,-1.015347,-1.031233,0.986465,-0.899709


train_target:


,Piauí - Consumo de Cimento (t)
0,536.766
1,639.535
2,625.022


test_input:


...

,Piauí - IDH Renda,Piauí - Desemprego,Piauí - IDH Educacao,Piauí - IDH Longevidade,Piauí - PIB - Estadual,Piauí - PIB - Construção Civil,Piauí - PIB - Per Capita,Piauí - PIB - Preços de Mercado,Piauí - Produção de Cimento (t),Contratação Comercial (Bi R$),...,NFSP - Porcentagem do PIB (%),Taxa Selic (%),Piauí - IDH,Precipitation (mm/day),Atmospheric Pressure(mB),Global Radiation - Daylight (Kj/m²),Air Temperature - Dry Bulb (°C),Dew Point Temperature (°C),Air Relative Humidity (%),Wind Gust (m/s)
12,0.823567,0.785725,1.126392,-0.598225,0.529639,0.378729,0.253155,0.276615,-1.651728,1.108405,...,-1.482779,-2.260447,0.91739,1.011919,0.418013,-0.843464,1.091606,-0.183281,-0.442768,-1.555022


test_target:


,Piauí - Consumo de Cimento (t)
12,826.368


1/1 [==============================] - 1s 554ms/step
Error: 51.8744086914063




[639.6097412109375,
 629.4610595703125,
 711.08740234375,
 674.9244995117188,
 794.6851196289062,
 831.9396362304688,
 828.1914672851562,
 724.3893432617188,
 710.4934692382812,
 774.4935913085938]

In [27]:
display(targets)
display(predictions)

[687.536,
 778.779,
 755.7138571428571,
 794.9454857142857,
 842.6876571428572,
 831.356,
 626.223,
 702.437,
 788.164,
 826.368]

[639.6097412109375,
 629.4610595703125,
 711.08740234375,
 674.9244995117188,
 794.6851196289062,
 831.9396362304688,
 828.1914672851562,
 724.3893432617188,
 710.4934692382812,
 774.4935913085938]

In [28]:
mae = mean_absolute_error(predictions, targets)
mae

76.39435639648438

In [29]:
porcentage = mae/np.mean(targets)
porcentage

0.10006845030001058